# Yasir Hassan
# k-means algorithm & PCA 
# For Anticline images

In [1]:
# import related libraries
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA

In [2]:
# getData function definition
def get_data(folder):
    #Load the data and labels from the given folder
    X = []
    y = []
    
    for seismic_type in os.listdir(folder):
             if not seismic_type.startswith('.'):
                 if seismic_type in ['Class1']:
                     label = '0'
                 else:
                     label = '1'
                 for image_filename in os.listdir(folder + seismic_type):
                     img_file = cv2.imread(folder + seismic_type + '/' + image_filename, 0)
                     if img_file is not None:
                         # Downsample the image to 120, 160, 3
                         # img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                         img_arr = np.asarray(img_file)
                         X.append(img_arr)
                         y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [3]:
# get data from the source
X, y = get_data("C:/Users/Input File1/")

In [4]:
# split the data randomnly
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size = 0.20, random_state = 0)

In [5]:
# convert X data from 3d to 2d array
nsamples, nx, ny = X_train.shape
nsamples2, nx2, ny2 = X_test.shape
X_train = X_train.reshape((nsamples,nx*ny))
X_test = X_test.reshape((nsamples2,nx2*ny2))

In [6]:
# convert y train and y test data to fit the classifier
nsamples  = y_train.shape
nsamples2 = y_test.shape
y_train = y_train.reshape(nsamples)
y_test = y_test.reshape(nsamples2)

In [7]:
# import transformer class - StandardScaler and fit it 
from sklearn.preprocessing import StandardScaler
stdScaler = StandardScaler()
stdScaler.fit(X_train)
X_train_scaled = stdScaler.transform(X_train)
X_test_scaled = stdScaler.transform(X_test)

In [8]:
# instantiate the KMeans for clustering and fit it with X train
# using two clusters, from domain knowledge.
kmeans = KMeans(n_clusters = 2, random_state=0)
kmeans.fit(X_train_scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [9]:
print("Cluster memberships:\n{}".format(kmeans.labels_))

Cluster memberships:
[0 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 0 0 1
 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 0 0 1 0 1 1 0
 1 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0
 1 0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 0
 0 1 1 1 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 1 1 0 1 1 0 1
 1 1 1 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 1 1 1 0
 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0
 1 0 1 1 0 1 0 1 1 1 0 1 0 1 0 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0
 1 1 1 0 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 1 0 0 1 1 0 1 0 1 1 0
 1 0 0 1 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 0]


In [10]:
y_test_pred = kmeans.predict(X_test_scaled)

In [11]:
# convert y_test from string to int (number)
y_test_num = [ int(item) for item in y_test]

In [12]:
# Compute confusion matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_num, y_test_pred)

In [13]:
print(cm)

[[35 11]
 [ 0 44]]


In [57]:
accuracy = accuracy_score(y_test_num, y_test_pred)
print(accuracy)

0.8777777777777778


In [14]:
# applying PCA
# keep the first two principal components of the data
pca = PCA(n_components = 2)

In [15]:
# fit PCA model with X_train_scaled
pca.fit(X_train_scaled)

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [16]:
# transform data onto the first two principal components
X_pca = pca.transform(X_train_scaled)
print("Original shape: {}".format(str(X_train_scaled.shape)))
print("Reduced shape: {}".format(str(X_pca.shape)))

Original shape: (360, 76800)
Reduced shape: (360, 2)


In [ ]:
# Evaluate the best Kmeans number of clusters:
# using Elbow method to evaluate the best Kmeans n_clusters.
# Elbow method gives an idea on what a good k number of clusters.
# Pick k at the spot where SSE starts to flatten out and 
# forming an elbow.
# SSE is the Sum of Squared Distance between data points 
# and their assigned clusters’ centroids.

sse = []
list_k = list(range(1, 10))

for k in list_k:
    km = KMeans(n_clusters=k)
    km.fit(X_train_scaled)
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance')

In [ ]:
# From the Elbow method above, n_clusters = 2 appeared to be the best choice.